## short tutorial on running MPI4py on EC2 

 First getting the information needed to start an EC2 cluster,
 I have my information stored in ~/.starcluster/config
 
 What is needed is: 
 * [security credintals](https://sites.google.com/site/emcloudprocessing/home/setting-up-running-a-cluster-on-amazon-ec2), that is acess_key_id aws_secret_key
 * [key_for_the_region](http://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-key-pairs.html)
 * and an [aws region](http://docs.aws.amazon.com/general/latest/gr/rande.html#ec2_region)

In [1]:
## short tutorial on running MPI4py on EC2 

from ConfigParser import ConfigParser
config = ConfigParser()
starfigconfig_folder = "/Users/jonaswallin/.starcluster/"
config.read(starfigconfig_folder + "config")

acess_key_id     = config.get('aws info', 'aws_access_key_id'    , 0)
aws_secret_key   = config.get('aws info', 'aws_secret_access_key', 0)
aws_region_name  = config.get('aws info', 'aws_region_name'      , 0)
my_key_loc       = config.get('key mykeyABC', 'key_location',0)

Now we setup the linkingEC2 object:

In [2]:
from linkingEC2 import LinkingHandler
linker = LinkingHandler(aws_secret_access_key = aws_secret_key,
                        aws_access_key_id     = acess_key_id,
                        aws_region_name       = aws_region_name,
                        key_location          = my_key_loc,
                        key_name              = 'mykeyABC' )
linker.start_cluster('ami-d05e75b8', 't2.micro', ['Deafult'],2)

waiting for instances 0  to be running: 
********************************************************

waiting for instances 1  to be running: 




NameError: global name 'my_key_loc' is not defined

In [3]:
linker.terminate_cluster()